In [ ]:
import pandas as pd
import geopandas as gpd
import random
from shapely.geometry import Point
from shapely.ops import unary_union

fdi_path = "/Users/amandaarrais/Desktop/MGC/In progress Datasets/FDI_dataset_final_coords.csv"
fdi_df = pd.read_csv(fdi_path)
fdi_df["Recipient Country"] = fdi_df["Recipient Country"].astype(str).str.strip()

world = gpd.read_file("/Users/amandaarrais/Desktop/countries.geojson")
world = world[["name", "geometry"]]
world["name"] = world["name"].astype(str).str.strip()

country_fixes = {
    "Bahamas": "The Bahamas",
    "Cape Verde": "Cabo Verde",
    "Congo [DRC]": "Democratic Republic of the Congo",
    "Congo [Republic]": "Republic of the Congo",
    "Côte d'Ivoire": "Ivory Coast",
    "Czech Republic": "Czechia",
    "Hong Kong": "Hong Kong S.A.R.",
    "Laos": "Laos",
    "Macedonia [FYROM]": "North Macedonia",
    "Myanmar [Burma]": "Myanmar",
    "Russia": "Russia",
    "São Tomé and Príncipe": "São Tomé and Principe",
    "Serbia": "Republic of Serbia",
    "Swaziland": "eSwatini",
    "Syria": "Syria",
    "Tanzania": "United Republic of Tanzania",
    "Timor-Leste": "East Timor",
    "Turkey": "Turkey",
    "United States": "United States of America",
    "Vietnam": "Vietnam"
}

fdi_df["Country_for_Map"] = fdi_df["Recipient Country"].replace(country_fixes)

merged = pd.merge(fdi_df, world, left_on="Country_for_Map", right_on="name", how="left")
gdf = gpd.GeoDataFrame(merged, geometry="geometry")

def get_random_point_in_country(geom):
    if geom is None:
        return None
    minx, miny, maxx, maxy = geom.bounds
    for _ in range(100):
        point = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if geom.contains(point):
            return point
    return geom.centroid  

gdf["RandomPoint"] = gdf["geometry"].apply(get_random_point_in_country)
gdf["Latitude"] = gdf["RandomPoint"].apply(lambda p: p.y if p else None)
gdf["Longitude"] = gdf["RandomPoint"].apply(lambda p: p.x if p else None)

final_df = gdf.drop(columns=["geometry", "name", "Country_for_Map", "RandomPoint"])
final_df.to_csv("/Users/amandaarrais/Desktop/MGC/In progress Datasets/FDI_dataset_random_within_country2.csv", index=False)

print("Random lat/lon generated inside country borders and saved.")
